In [1]:
%pip install tensorflow
%pip install pandas 
%pip install numpy

Note: you may need to restart the kernel to use updated packages.Requirement already satisfied: tensorflow in c:\users\bruker\appdata\local\programs\python\python311\lib\site-packages (2.17.0)

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
%load_ext autoreload

In [50]:
%autoreload 2
import tensorflow as tf
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
import datetime
import os
from feature_engineering.tensor_features import develop_features, floating_conv
from sklearn.preprocessing import StandardScaler
from data_handler import LocalToLargeDataLoader
from path_finder import path_sorter

In [7]:
data_loader = LocalToLargeDataLoader(print_progress=True)
parsed_data = data_loader.load_raw_data(path="../../resources")

Retrieving training data...


In [5]:
print(parsed_data.keys())

Index(['time', 'cog', 'sog', 'rot', 'heading', 'navstat', 'latitude',
       'longitude', 'vesselId', 'portId', 'etaParsed'],
      dtype='object')


In [6]:
path_dict = path_sorter(parsed_data)

[===================-] 100.0% complete
[====================] 100.0% complete
Concatting dataframes


In [14]:

print(list(path_dict.keys()))
print(path_dict["time_key"][0])
print(path_dict['navstat'])
print(path_dict[list(path_dict.keys())[0]][0])

['rot', 'heading', 'navstat', 'vesselId', 'portId', 'etaParsed', 'time_key', 'group_id', 'time_0', 'cog_0', 'sog_0', 'latitude_0', 'longitude_0', 'time_1', 'cog_1', 'sog_1', 'latitude_1', 'longitude_1', 'time_2', 'cog_2', 'sog_2', 'latitude_2', 'longitude_2', 'time_3', 'cog_3', 'sog_3', 'latitude_3', 'longitude_3', 'time_4', 'cog_4', 'sog_4', 'latitude_4', 'longitude_4', 'time_5', 'cog_5', 'sog_5', 'latitude_5', 'longitude_5', 'time_6', 'cog_6', 'sog_6', 'latitude_6', 'longitude_6', 'time_7', 'cog_7', 'sog_7', 'latitude_7', 'longitude_7', 'time_8', 'cog_8', 'sog_8', 'latitude_8', 'longitude_8', 'time_9', 'cog_9', 'sog_9', 'latitude_9', 'longitude_9']
2024-05-05 13:25:50
0         1
1         0
2         0
3         0
4         0
         ..
152527    0
152528    0
152529    5
152530    0
152531    0
Name: navstat, Length: 152532, dtype: int64
9


In [40]:
features, y, test_features, y_test = data_loader.load_training_data(path_dict) 

Sorting into tests and training data


In [41]:
features=develop_features(features)
test_features=develop_features(test_features)

Developing features with the following columns:  Index(['rot', 'heading', 'navstat', 'etaParsed', 'time_0', 'time_1', 'cog_1',
       'sog_1', 'latitude_1', 'longitude_1', 'time_2', 'cog_2', 'sog_2',
       'latitude_2', 'longitude_2', 'time_3', 'cog_3', 'sog_3', 'latitude_3',
       'longitude_3', 'time_4', 'cog_4', 'sog_4', 'latitude_4', 'longitude_4',
       'time_5', 'cog_5', 'sog_5', 'latitude_5', 'longitude_5', 'time_6',
       'cog_6', 'sog_6', 'latitude_6', 'longitude_6', 'time_7', 'cog_7',
       'sog_7', 'latitude_7', 'longitude_7', 'time_8', 'cog_8', 'sog_8',
       'latitude_8', 'longitude_8', 'time_9', 'cog_9', 'sog_9', 'latitude_9',
       'longitude_9'],
      dtype='object')
Developing features with the following columns:  Index(['rot', 'heading', 'navstat', 'etaParsed', 'time_0', 'time_1', 'cog_1',
       'sog_1', 'latitude_1', 'longitude_1', 'time_2', 'cog_2', 'sog_2',
       'latitude_2', 'longitude_2', 'time_3', 'cog_3', 'sog_3', 'latitude_3',
       'longitude_3', 

In [49]:
print(list(features.keys()))
print(len(list(features.keys())))
# print(features.head())
print(y.head())

['rot', 'heading', 'navstat', 'etaParsed', 'time_0', 'time_1', 'cog_1', 'sog_1', 'latitude_1', 'longitude_1', 'time_2', 'cog_2', 'sog_2', 'latitude_2', 'longitude_2', 'time_3', 'cog_3', 'sog_3', 'latitude_3', 'longitude_3', 'time_4', 'cog_4', 'sog_4', 'latitude_4', 'longitude_4', 'time_5', 'cog_5', 'sog_5', 'latitude_5', 'longitude_5', 'time_6', 'cog_6', 'sog_6', 'latitude_6', 'longitude_6', 'time_7', 'cog_7', 'sog_7', 'latitude_7', 'longitude_7', 'time_8', 'cog_8', 'sog_8', 'latitude_8', 'longitude_8', 'time_9', 'cog_9', 'sog_9', 'latitude_9', 'longitude_9']
50
        latitude_0  longitude_0
39974     49.16460   -123.92530
3942      13.06485    100.88147
14723     32.90688    -79.95385
73623     41.53984    140.65908
100269     1.31259    104.32622


In [51]:

X_train = features.to_numpy()
X_test_np = test_features.to_numpy()
y_train = y.to_numpy()
y_test_np = y_test.to_numpy()


In [52]:
model = Sequential()

# Input layer
input_dim = X_train.shape[1]  # Number of features in your dataset

# Add layers
model.add(Dense(64, input_dim=input_dim, activation='relu'))  # First hidden layer
model.add(Dropout(0.3))  # Dropout for regularization
model.add(Dense(32, activation='relu'))  # Second hidden layer
model.add(Dense(16, activation='relu'))  # Third hidden layer

# Output layer
# If it's a regression task (predicting a continuous variable like time):
model.add(Dense(units=2, activation='linear')) 

model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error', metrics=['mae'])

c:\Users\Bruker\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [1]:
# parameters=(input_shape[0])
# print(parameters)
# layer_n=2
# layer=[
#         tf.keras.layers.Normalization(),
#         tf.keras.layers.Dense(parameters//layer_n+1, activation='relu'),
#         tf.keras.layers.Dropout(0.2)
#         ]

# model_list=[tf.keras.Input(shape=input_shape),
#             tf.keras.layers.Dense(input_shape[0], activation='relu')]
# model_list.extend(layer*layer_n)
# model_list.append(tf.keras.layers.Dense(2, activation='linear'))
# print(model_list)
# model = tf.keras.models.Sequential(model_list)

# model.compile(optimizer='adam',
#   loss='mean_squared_error',
#   metrics=['mae','mape'])

# model.summary()

# early_stopping = EarlyStopping(
#     monitor='val_loss',  # You can change this to 'val_accuracy' or another metric
#     patience=5,          # Number of epochs to wait for improvement
#     restore_best_weights=True  # Restore model weights from the epoch with the best value of the monitored quantity
# )

# checkpoint = ModelCheckpoint(
#     filepath='modelepoch{epoch:02d}val_loss{val_loss:.2f}.keras',
#     monitor='val_loss',
#     save_best_only=True,
#     save_weights_only=False,
#     mode='min',
#     verbose=1
# )

# log_dir= os.path.join('logs','fit',datetime.datetime.now().strftime("%Y%m%d-%H%M%S"),)
# # tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)


NameError: name 'input_shape' is not defined

In [53]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# Define callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
model_checkpoint = ModelCheckpoint('best_model.keras', save_best_only=True)


ValueError: The filepath provided must end in `.keras` (Keras model format). Received: filepath=best_model.h5

In [26]:
history = model.fit(X_train, y_train, epochs=100, batch_size=32, 
                    validation_split=0.2, callbacks=[early_stopping, model_checkpoint])

Y data samples [[  36.84686    5.84638]
 [  48.5332    -6.12003]
 [  28.82071 -120.46513]
 [  33.86918 -118.81433]
 [  33.18068   -9.43055]]
Epoch 1/20
 8/14 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 28622121336832.0000 - mae: 4240309.5000 - mape: 38419712.0000  
Epoch 1: val_loss did not improve from 23641094144.00000
14/14 ━━━━━━━━━━━━━━━━━━━━ 1s 29ms/step - loss: 25887286231040.0000 - mae: 4034683.5000 - mape: 36272012.0000 - val_loss: 775497777152.0000 - val_mae: 811846.8125 - val_mape: 10468310.0000
Epoch 2/20
 1/14 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step - loss: 22770285019136.0000 - mae: 4008617.0000 - mape: 17482132.0000
Epoch 2: val_loss did not improve from 23641094144.00000
14/14 ━━━━━━━━━━━━━━━━━━━━ 1s 41ms/step - loss: 18431774556160.0000 - mae: 3425951.7500 - mape: 25047640.0000 - val_loss: 110672863232.0000 - val_mae: 282489.5000 - val_mape: 3566033.5000
Epoch 3/20
13/14 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 10771739181056.0000 - mae: 2604610.0000 - mape: 21291736.0000  
Epo

In [ ]:
loss, mae = model.evaluate(X_test_np, y_test)
print(f"Test Loss: {loss}, Test MAE: {mae}")